# Introduction to GrammarParsingTagger

GrammarParsingTagger is a tool that allows us to write a context-free grammar and apply the grammar on our Text object, creating a new layer that contains all the matches found by the grammar. This means that we can define the sequences of symbols that we want to extract from the text. Let's look at an example of extracting addresses from a text.

First, we need to have our Text object. Let's create one containing an address:

In [1]:
from estnltk import Text

text = Text('Jüri Homenja kontsert toimub E, 22. mai kl 18:00 kultuurimajas Veski 5, Elva, Tartumaa.')

In [2]:
text

text
"Jüri Homenja kontsert toimub E, 22. mai kl 18:00 kultuurimajas Veski 5, Elva, Tartumaa."


Next, we need to tag the symbols whose sequences we want to search with our grammar. You can find out about different taggers from [here](https://github.com/estnltk/estnltk/tree/devel_1.6/tutorials/taggers), but for the GrammarParsingTagger example, let's not dive into this but use an existing tagger called AddressPartTagger. AddressPartTagger needs the text to be segmented into words, that's why we tag the layer *words* on the text before applying the tagger.

In [3]:
from estnltk.taggers import AddressPartTagger

address_part_tagger = AddressPartTagger()
text.tag_layer(['words'])
address_part_tagger.tag(text)
text.address_parts

Layer(name=address_parts, spans=SL[AS[Span(, {'grammar_symbol': 'RANDOM_TEXT', 'type': None})],
AS[Span(, {'grammar_symbol': 'RANDOM_TEXT', 'type': None})],
AS[Span(Jüri, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Jüri, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Jüri, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(22, {'grammar_symbol': 'MAJA', 'type': None})],
AS[Span(18, {'grammar_symbol': 'MAJA', 'type': None})],
AS[Span(00, {'grammar_symbol': 'MAJA', 'type': None})],
AS[Span(kultuurimajas, {'grammar_symbol': 'RANDOM_TEXT', 'type': None})],
AS[Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(5, {'grammar_symbol': 'MAJA', 'type': None})],
AS[Span(Elva, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Elva, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(Tartumaa, {'grammar_symbol': 'MAAKOND', 'type': 'maakond'})]])

Now, we can see that we have different symbols tagged on the text in the layer called *address_parts*. Some of the tagged symbols are in fact parts of the address, but others are not. To know whether the symbol is part of an address, we have to define the sequences of symbols that make up an address. These sequences are called grammar **rules**. 

## Rules and Grammar

To define rules and a grammar, we first need to import the classes Rule and Grammar.

In [4]:
from estnltk.finite_grammar import Rule, Grammar

Then it is possible to start defining the rules. A rule consists of a left side (non-terminal), a right side (non-terminals and terminals), and optional parameters. In the following example of a rule, the left side is 'ADDRESS', the right side is 'TÄNAV MAJA ASULA', and there are optional parameters *group* and *priority*. The rule says that if symbols 'TÄNAV', 'MAJA', and 'ASULA' occur in this order, this is an 'ADDRESS'. The parameter *group* is a name for the group into which the rule belongs, it can be anything, but all rules that we want to put into one group need to have the same name for *group*. *priority* defines which rule of the ones belonging to the *group* is applied if several rules match. **NB!** The **higher** the value, the **lower** the priority. Therefore, if a there are two rules with priorities 2 and 3 that both match, the rule with the priority of 2 is applied.

In [5]:
Rule('ADDRESS', 'TÄNAV MAJA ASULA', group='g0', priority=3)

ADDRESS -> TÄNAV MAJA ASULA	: 3, val: default_validator, dec: default_decorator, scoring: default_scoring

To apply the rules, we need to create a grammar:

In [6]:
grammar = Grammar(start_symbols=['ADDRESS'], 
                  depth_limit=float('inf'), # the default
                  width_limit=float('inf'), # the default
                  legal_attributes=None # the default
                  )

And then we need to add rules to the grammar. Let's first add two rules that belong to the same group and have equal priorities:

In [7]:
grammar.add_rule('ADDRESS', 'TÄNAV MAJA ASULA', group='g0', priority=3)
grammar.add_rule('ADDRESS', 'TÄNAV MAJA',       group='g0', priority=3)
grammar


Grammar:
	start: ADDRESS
	terminals: ASULA, MAJA, TÄNAV
	nonterminals: ADDRESS
	legal attributes: frozenset()
	depth_limit: inf
	width_limit: inf
Rules:
	ADDRESS -> TÄNAV MAJA ASULA	: 3, val: default_validator, dec: default_decorator, scoring: default_scoring
	ADDRESS -> TÄNAV MAJA	: 3, val: default_validator, dec: default_decorator, scoring: default_scoring

To apply the grammar on our text, we need to create a tagger - a GrammarParsingTagger object. This tagger gets our grammar for the parameter *grammar*. *layer_of_tokens* is the name of the layer that we want to apply our grammar on and *layer_name* is the name of the layer that we are creating with the GrammarParsingTagger. 

In [8]:
from estnltk.taggers import GrammarParsingTagger

tagger = GrammarParsingTagger(grammar=grammar,
                              layer_of_tokens='address_parts',
                              layer_name='addresses', # default: 'parse'
                              output_ambiguous=True # default False, True recommended
                              )
tagger

GrammarParsingTagger(input_layers=['address_parts'], output_layer=addresses, output_attributes=(), grammar=
Grammar:
	start: ADDRESS
	terminals: ASULA, MAJA, TÄNAV
	nonterminals: ADDRESS
 ..., type: <class 'estnltk.finite_grammar.grammar.Grammar'>, name_attribute=grammar_symbol, output_nodes={'ADDRESS'}, resolve_support_conflicts=True, resolve_start_end_conflicts=True, resolve_terminals_conflicts=True, ambiguous=True, gap_validator=None, debug=False)

Then we can use the tagger to tag the text:

In [9]:
tagger.tag(text)

Text(text="Jüri Homenja kontsert toimub E, 22. mai kl 18:00 kultuurimajas Veski 5, Elva, Tartumaa.")

In [10]:
text.addresses

Layer(name=addresses, spans=SL[AS[ES[AS[Span(Jüri, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Jüri, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Jüri, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(22, {'grammar_symbol': 'MAJA', 'type': None})]]],
AS[ES[AS[Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(5, {'grammar_symbol': 'MAJA', 'type': None})]]],
AS[ES[AS[Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(5, {'grammar_symbol': 'MAJA', 'type': None})],
AS[Span(Elva, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Elva, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})]]]])

The first problem that we see is that 'Jüri', '22' has been tagged as address, although the tokens are not even next to each other in the original text. However, as the grammar is looking at the layer *address_parts* and there's nothing between the grammar_symbols of these tokens, they are tagged as an address. To overcome this problem, we can use a gaps_validator function where we can define what kind of gaps we allow between our tagged symbols - e.g. we would probably want to accept a space between the parts of an address but not long sequences of words or sentences. 

# Priorities

In [11]:
grammar = Grammar(start_symbols=['ADDRESS'], 
                  rules=None, # the default, deprecated
                  depth_limit=float('inf'), # the default
                  width_limit=float('inf'), # the default
                  legal_attributes=None # the default
                  )

grammar.add_rule('ADDRESS', 'TÄNAV MAJA ASULA', group='g0', priority=2)
grammar.add_rule('ADDRESS', 'TÄNAV MAJA',       group='g0', priority=3)
tagger = GrammarParsingTagger(grammar=grammar,
                              layer_of_tokens='address_parts',
                              name_attribute='grammar_symbol', # the default
                              layer_name='addresses_2', # the default
                              attributes=(), # default: ()
                              output_ambiguous=True # default False
                              )
tagger.tag(text)
text.addresses_2

Layer(name=addresses_2, spans=SL[AS[ES[AS[Span(Jüri, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Jüri, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Jüri, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(22, {'grammar_symbol': 'MAJA', 'type': None})]]],
AS[ES[AS[Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(5, {'grammar_symbol': 'MAJA', 'type': None})],
AS[Span(Elva, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Elva, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})]]]])

# Decorators

In [12]:
def address_decorator(nodes):
    asula = ''
    maakond = ''
    t2nav = ''
    indeks = ''
    maja = ''
    for node in nodes:
        if node.name == 'ASULA':
            asula = node.text#[0]
        elif node.name == 'TÄNAV':
            t2nav = node.text#[0]
        elif node.name == 'MAAKOND':
            maakond = node.text#[0]
        elif node.name == 'MAJA':
            maja = node.text#[0]
        elif node.name == 'INDEKS':
            indeks = node.text#[0]
    return {'grammar_symbol': 'ADDRESS',
            'ASULA': asula,
            'TÄNAV': t2nav,
            'INDEKS': indeks,
            'MAAKOND': maakond,
            'MAJA': maja}

grammar = Grammar(start_symbols=['ADDRESS'], 
                  rules=None, # the default, deprecated
                  depth_limit=float('inf'), # the default
                  width_limit=float('inf'), # the default
                  legal_attributes=['INDEKS', 'grammar_symbol', 'MAJA', 'TÄNAV', 'MAAKOND', 'ASULA']
                  )

grammar.add_rule('ADDRESS', 'TÄNAV MAJA ASULA', group='g0', priority=3, decorator=address_decorator)
grammar.add_rule('ADDRESS', 'TÄNAV MAJA',       group='g0', priority=3, decorator=address_decorator)
tagger = GrammarParsingTagger(grammar=grammar,
                              layer_of_tokens='address_parts',
                              name_attribute='grammar_symbol',
                              layer_name='addresses_3',
                              attributes=('INDEKS', 'grammar_symbol', 'MAJA', 'TÄNAV', 'MAAKOND', 'ASULA'),
                              output_nodes=None,
                              resolve_support_conflicts=True,
                              resolve_start_end_conflicts=True,
                              resolve_terminals_conflicts=True,
                              output_ambiguous=False # default False
                              )
tagger.tag(text)
text.addresses_3

Layer(name=addresses_3, spans=SL[ES[AS[Span(Jüri, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Jüri, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Jüri, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(22, {'grammar_symbol': 'MAJA', 'type': None})]],
ES[AS[Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(5, {'grammar_symbol': 'MAJA', 'type': None})]],
ES[AS[Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(5, {'grammar_symbol': 'MAJA', 'type': None})],
AS[Span(Elva, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Elva, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})]]])

# Validators

In [13]:
text = Text('Inimesed, kes töötavad Tartus Ülikooli 5, Elva haiglas \
            ja Tõravere observatooriumis, söövad esmaspäeviti õunu.').tag_layer(['words'])
address_part_tagger.tag(text)

grammar = Grammar(start_symbols=['ADDRESS'],
                  legal_attributes=['INDEKS', 'grammar_symbol', 'MAJA', 'TÄNAV', 'MAAKOND', 'ASULA']
                  )

grammar.add_rule('ADDRESS', 'TÄNAV MAJA ASULA', group='g0', priority=3, decorator=address_decorator)
grammar.add_rule('ADDRESS', 'TÄNAV MAJA',       group='g0', priority=3, decorator=address_decorator)
tagger = GrammarParsingTagger(grammar=grammar,
                              layer_of_tokens='address_parts',
                              name_attribute='grammar_symbol',
                              layer_name='addresses_4',
                              attributes=('INDEKS', 'grammar_symbol', 'MAJA', 'TÄNAV', 'MAAKOND', 'ASULA')
                              )
tagger.tag(text)
text.addresses_4

Layer(name=addresses_4, spans=SL[ES[AS[Span(Ülikooli, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(5, {'grammar_symbol': 'MAJA', 'type': None})]],
ES[AS[Span(Ülikooli, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(5, {'grammar_symbol': 'MAJA', 'type': None})],
AS[Span(Elva, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Elva, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})]]])

In [14]:
text = Text('Inimesed, kes töötavad Tartus Ülikooli 5, Elva haiglas \
            ja Tõravere observatooriumis, söövad esmaspäeviti õunu.').tag_layer(['words'])
address_part_tagger.tag(text)

town_streets = {'Elva': {'Veski', 'Tuletõrje'},
                'Tartu': {'Veski', 'Ülikooli'}}

def validator(node):
    street = node[0].text
    town = node[2].text
    if town in town_streets:
        return street in town_streets[town]
    return True

grammar = Grammar(start_symbols=['ADDRESS'], 
                  legal_attributes=['INDEKS', 'grammar_symbol', 'MAJA', 'TÄNAV', 'MAAKOND', 'ASULA']
                  )

grammar.add_rule('ADDRESS', 'TÄNAV MAJA ASULA', group='g0', priority=3, decorator=address_decorator, validator=validator)
grammar.add_rule('ADDRESS', 'TÄNAV MAJA',       group='g0', priority=3, decorator=address_decorator)
tagger = GrammarParsingTagger(grammar=grammar,
                              layer_of_tokens='address_parts',
                              name_attribute='grammar_symbol',
                              layer_name='addresses_4',
                              attributes=('INDEKS', 'grammar_symbol', 'MAJA', 'TÄNAV', 'MAAKOND', 'ASULA'),
                              output_ambiguous=True
                              )
tagger.tag(text)
text.addresses_4

Layer(name=addresses_4, spans=SL[AS[ES[AS[Span(Ülikooli, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(5, {'grammar_symbol': 'MAJA', 'type': None})]]]])

## `SEQ` rules

In [15]:
def address_decorator(nodes):
    asula = ''
    maakond = ''
    t2nav = ''
    indeks = ''
    maja = ''
    for node in nodes:
        if node.name == 'ASULA':
            asula = node.text
        elif node.name == 'TÄNAV':
            t2nav = node.text
        elif node.name == 'MAAKOND':
            maakond = node.text
        elif node.name == 'MAJA':
            maja = node.text
        elif node.name == 'SEQ(MAJA)':
            maja = [n.text for n in node.support]
        elif node.name == 'INDEKS':
            indeks = node.text
    return {'grammar_symbol': 'ADDRESS',
            'ASULA': asula,
            'TÄNAV': t2nav,
            'INDEKS': indeks,
            'MAAKOND': maakond,
            'MAJA': maja}

text = Text('Veekatkestus Tartu Veski tänava majades 3, 5, 7.').tag_layer(['words'])
address_part_tagger.tag(text)

grammar = Grammar(start_symbols=['ADDRESS'],
                  legal_attributes=['INDEKS', 'grammar_symbol', 'MAJA', 'TÄNAV', 'MAAKOND', 'ASULA']
                  )
def scoring(node):
    return len(node[2].support)

grammar.add_rule('ADDRESS', 'ASULA TÄNAV SEQ(MAJA)', group='g0', priority=3, decorator=address_decorator, scoring=scoring)
tagger = GrammarParsingTagger(grammar=grammar,
                              layer_of_tokens='address_parts',
                              name_attribute='grammar_symbol',
                              layer_name='addresses_4',
                              attributes=('INDEKS', 'grammar_symbol', 'MAJA', 'TÄNAV', 'MAAKOND', 'ASULA'),
                              output_ambiguous=True
                              )
tagger.tag(text)
text.addresses_4

Layer(name=addresses_4, spans=SL[AS[ES[AS[Span(Tartu, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Tartu, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(3, {'grammar_symbol': 'MAJA', 'type': None})]]],
AS[ES[AS[Span(Tartu, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Tartu, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(3, {'grammar_symbol': 'MAJA', 'type': None})],
AS[Span(5, {'grammar_symbol': 'MAJA', 'type': None})]]],
AS[ES[AS[Span(Tartu, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Tartu, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(3, {'grammar_symbol': 'MAJA', 'type': None})],
AS[Span(5, {'grammar_symbol': 'MAJA', 'type': None})],
AS[Span(7, {'grammar_symbol': 'MAJA', 'type': None})]]]])

## MSEQ rules
MSEQ rule matches the maximal of the symbol sequences.

In [16]:
def address_decorator(nodes):
    asula = ''
    maakond = ''
    t2nav = ''
    indeks = ''
    maja = ''
    for node in nodes:
        if node.name == 'ASULA':
            asula = node.text#[0]
        elif node.name == 'TÄNAV':
            t2nav = node.text#[0]
        elif node.name == 'MAAKOND':
            maakond = node.text#[0]
        elif node.name == 'MAJA':
            maja = node.text#[0]
        elif node.name == 'MSEQ(MAJA)':
            maja = [n.text for n in node.support]
        elif node.name == 'INDEKS':
            indeks = node.text#[0]
    return {'grammar_symbol': 'ADDRESS',
            'ASULA': asula,
            'TÄNAV': t2nav,
            'INDEKS': indeks,
            'MAAKOND': maakond,
            'MAJA': maja}

text = Text('Veekatkestus Tartu Veski tänava majades 3, 5, 7.').tag_layer(['words'])
address_part_tagger.tag(text)

grammar = Grammar(start_symbols=['ADDRESS'],
                  legal_attributes=['INDEKS', 'grammar_symbol', 'MAJA', 'TÄNAV', 'MAAKOND', 'ASULA']
                  )
def scoring(node):
    return len(node[2].support)

grammar.add_rule('ADDRESS', 'ASULA TÄNAV MSEQ(MAJA)', group='g0', priority=3, decorator=address_decorator, scoring=scoring)
tagger = GrammarParsingTagger(grammar=grammar,
                              layer_of_tokens='address_parts',
                              name_attribute='grammar_symbol',
                              layer_name='addresses_4',
                              attributes=('INDEKS', 'grammar_symbol', 'MAJA', 'TÄNAV', 'MAAKOND', 'ASULA'),
                              output_ambiguous=True
                              )
tagger.tag(text)
text.addresses_4

Layer(name=addresses_4, spans=SL[AS[ES[AS[Span(Tartu, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Tartu, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'ASULA', 'type': 'asula'}), Span(Veski, {'grammar_symbol': 'TÄNAV', 'type': 'tänav'})],
AS[Span(3, {'grammar_symbol': 'MAJA', 'type': None})],
AS[Span(5, {'grammar_symbol': 'MAJA', 'type': None})],
AS[Span(7, {'grammar_symbol': 'MAJA', 'type': None})]]]])